In [1]:
%pip install gensim 

Note: you may need to restart the kernel to use updated packages.


In [2]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [28]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
# import spacy
from gensim.models import Word2Vec
import gensim
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,LSTM,Dense,SimpleRNN,Dropout, Dropout, BatchNormalization,Bidirectional,GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
import re
nltk.download('stopwords')
nltk.download('punkt')
nltk.download("wordnet")
# nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
train = pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip',sep="\t")
test = pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip',sep="\t")
test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [5]:
train.shape

(156060, 4)

In [6]:
test.shape

(66292, 3)

In [7]:
train.isna().sum()

PhraseId      0
SentenceId    0
Phrase        0
Sentiment     0
dtype: int64

In [8]:
test.isna().sum()

PhraseId      0
SentenceId    0
Phrase        1
dtype: int64

In [9]:
train.duplicated().sum()

0

In [10]:
len(train[train['PhraseId']==1])

1

In [11]:
train.iloc[:65,:]

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
60,61,1,a story,2
61,62,1,story,2
62,63,1,.,2
63,64,2,"This quiet , introspective and entertaining in...",4


## Number of Sentences

In [12]:
train.SentenceId.nunique()

8529

## Spliting Data

In [13]:
output = train.Sentiment.values
output= np.array(output)

# Cleaning Text

### Method2

In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66292 entries, 0 to 66291
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PhraseId    66292 non-null  int64 
 1   SentenceId  66292 non-null  int64 
 2   Phrase      66291 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [15]:
test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [16]:
def clean_sentences(df):
    reviews = []
    for sent in df['Phrase']:       
        if pd.isna(sent):
            print('a33333333333333333333333333333333333333')
            reviews.append('<EMPTY>')
            continue
        review_text = re.sub("[^a-zA-Z]"," ", sent)
        
        words = word_tokenize(review_text.lower())
        
        lemmatizer = WordNetLemmatizer()
        lemma_words = [lemmatizer.lemmatize(i) for i in words]
        
        reviews.append(lemma_words)
    return(reviews)


In [17]:
train_cleaned_text = clean_sentences(train)
test_cleaned_text = clean_sentences(test)

a33333333333333333333333333333333333333


In [18]:
print(len(train_cleaned_text))
print(len(test_cleaned_text))

156060
66292


In [19]:
print(train_cleaned_text[0])
print(test_cleaned_text[0])

['a', 'series', 'of', 'escapade', 'demonstrating', 'the', 'adage', 'that', 'what', 'is', 'good', 'for', 'the', 'goose', 'is', 'also', 'good', 'for', 'the', 'gander', 'some', 'of', 'which', 'occasionally', 'amuses', 'but', 'none', 'of', 'which', 'amount', 'to', 'much', 'of', 'a', 'story']
['an', 'intermittently', 'pleasing', 'but', 'mostly', 'routine', 'effort']


In [20]:
train_unique_words = set()
train_lengths = 0
test_unique_words = set()
test_lengths = 0
for word in train_cleaned_text:
    train_unique_words.update(word)
    train_lengths = max(train_lengths,len(word))
for word in test_cleaned_text:
    test_unique_words.update(word)
    test_lengths = max(test_lengths,len(word))

In [21]:
print("NUM of unique words in Train",len(train_unique_words))
print("NUM of unique words in Test",len(test_unique_words))
print('#########################')
print('Longest word in Train',train_lengths)
print('Longest word in Train',test_lengths)

NUM of unique words in Train 13745
NUM of unique words in Test 8730
#########################
Longest word in Train 48
Longest word in Train 52


# Text Vectorization

### word Embedding Layer

In [22]:
tokenizer=Tokenizer(num_words=len(train_unique_words),oov_token='<OOV>')
tokenizer.fit_on_texts(train_cleaned_text)
input_train=tokenizer.texts_to_sequences(train_cleaned_text)
input_train=pad_sequences(input_train,maxlen=50)

input_test = tokenizer.texts_to_sequences(test_cleaned_text)
input_test=pad_sequences(input_test, maxlen=50)

In [23]:
input_train.shape

(156060, 50)

# Model

In [24]:
model=Sequential()

model.add(Embedding(input_dim=13745,output_dim=300,input_length=input_train.shape[1]))

model.add(GRU(256, dropout = 0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(GRU(128,dropout = 0.2,recurrent_dropout=0.3,return_sequences=True))
model.add(GRU(128,return_sequences=False))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5,activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [25]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [26]:
model.compile(optimizer='adam', loss='SparseCategoricalCrossentropy', metrics=['accuracy'])

In [31]:
early_stopping = EarlyStopping(min_delta = 0.001,
                               mode = 'max',
                               monitor = 'val_accuracy',
                               patience = 2,
                              restore_best_weights=True)
callback = [early_stopping]

In [32]:
model.fit(input_train,output,epochs=15,batch_size=512,callbacks=[early_stopping],validation_split=.3)

Epoch 1/15
214/214 ━━━━━━━━━━━━━━━━━━━━ 38s 177ms/step - accuracy: 0.6955 - loss: 0.7442 - val_accuracy: 0.5990 - val_loss: 0.9924
Epoch 2/15
214/214 ━━━━━━━━━━━━━━━━━━━━ 38s 176ms/step - accuracy: 0.7205 - loss: 0.6786 - val_accuracy: 0.5988 - val_loss: 1.0074
Epoch 3/15
214/214 ━━━━━━━━━━━━━━━━━━━━ 38s 176ms/step - accuracy: 0.7301 - loss: 0.6472 - val_accuracy: 0.6002 - val_loss: 1.0056
Epoch 4/15
214/214 ━━━━━━━━━━━━━━━━━━━━ 37s 175ms/step - accuracy: 0.7419 - loss: 0.6167 - val_accuracy: 0.5936 - val_loss: 1.0448
Epoch 5/15
214/214 ━━━━━━━━━━━━━━━━━━━━ 38s 176ms/step - accuracy: 0.7507 - loss: 0.5922 - val_accuracy: 0.5993 - val_loss: 1.0412


In [33]:
input_test.shape

(66292, 50)

In [34]:
predicted = model.predict(input_test)

2072/2072 ━━━━━━━━━━━━━━━━━━━━ 71s 34ms/step


In [37]:
cls_predited=np.argmax(predicted,axis=-1)
cls_predited

array([2, 2, 2, ..., 2, 2, 1])

In [38]:
print(len(test['PhraseId']))
print(len(cls_predited))

66292
66292


In [40]:
submission = pd.DataFrame ({
    "PhraseId": test['PhraseId'],
    "Sentiment":cls_predited})
submission.to_csv("submission.csv", index=False)
submission.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2
